In [1]:
import pandas as pd
import tensorflow as tf
import requests
import json
import base64
import random

In [2]:
df = pd.read_csv("data/data.csv")
df["statement"]

0                                               oh my gosh
1        trouble sleeping, confused mind, restless hear...
2        All wrong, back off dear, forward doubt. Stay ...
3        I've shifted my focus to something else but I'...
4        I'm restless and restless, it's been a month n...
                               ...                        
51088    Anxiety cause faintness when standing up ? As ...
51089    anxiety heart symptom does anyone else have th...
51090    Travel Anxiety Hi all! Long time anxiety suffe...
51091    fomo from things i’m not involved in does anyo...
51092    Getting through the day How do you get  anythi...
Name: statement, Length: 51093, dtype: object

In [7]:
sampleNumber = random.randint(0, len(df) - 1)
textSample = df["statement"][sampleNumber]
print("Sample text:", textSample)

Sample text: I like skincare that much, so I'm happy with any brand BA disociolla


In [8]:
def prepare_json(text):
    feature_spec = {
        "statement": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(textSample)

endpoint = "https://dicoding-deploy-tfx-production.up.railway.app/v1/models/mental-health-model:predict"
response = requests.post(endpoint, data=json_data)

if response:
    probabilities = response.json().get("predictions")[0]
    
    max_index = probabilities.index(max(probabilities))
    
    labels = [
        'Anxiety', 'Bipolar', 'Depression', 'Normal', 
        'Personality disorder', 'Stress', 'Suicidal'
    ]
    
    result = labels[max_index]
    confidence = max(probabilities)
    
    print(f"Prediction: {result} (Confidence: {confidence:.2f})")
else:
    result = "Error: No predictions found in response."
    print(result)

Prediction: Normal (Confidence: 1.00)
